In [ ]:
### FOR COLAB
import os
is_colab = 'google.colab' in str(get_ipython())
if is_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    colab_path = '/content/drive/MyDrive/HumbugDB new/HumBugDB/notebooks'
    if not os.getcwd().endswith('ish'):
      os.chdir(colab_path)
    ## DO VERSION CORRECTING STUFF
    # TODO
import sys
sys.path.insert(0, os.path.abspath('../torchvggish/torchvggish'))
sys.path.append('../lib')

In [ ]:
import torch
import pandas as pd
import numpy as np
import config
from sklearn.utils import shuffle, class_weight
from tqdm import tqdm

In [ ]:
import mel_features

In [ ]:
import torch

model = torch.hub.load('harritaylor/torchvggish', 'vggish')
model.eval()

# Download an example audio file
import urllib
url, filename = ("http://soundbible.com/grab.php?id=1698&type=wav", "bus_chatter.wav")
file_dir = '../torchvggish'
try: urllib.URLopener().retrieve(url, os.path.join(file_dir, filename))
except: urllib.request.urlretrieve(url, os.path.join(file_dir, filename))

bus_chatter_out = model.forward(os.path.join(file_dir, filename))

In [ ]:
bus_chatter_out.shape

In [ ]:
test_out = model.forward('test.wav')
test_out.shape

In [ ]:
model.forward(filename)

In [ ]:
from vggish_input import wavfile_to_examples

In [ ]:
log_mel_examples = wavfile_to_examples('test.wav') 
bus_feat = wavfile_to_examples('bus_chatter.wav') 

In [ ]:
print(log_mel_examples.shape, bus_feat.shape)

In [ ]:
print(model)

In [ ]:
test_feat_output = model.features(log_mel_examples)
bus_feat_output = model.features(bus_feat)


In [ ]:
print(test_feat_output.shape, bus_feat_output.shape)

So we need one extra layer on top of this (or possibly remove the last layer first) that gets it down to a 8-class classifier

In [ ]:
from runTorch import train_model 
test_X_train = test_feat_output # as in from test.wav not tes data
test_y_train = np.random.randint(8, size=(4,)) # numpy array not torch tensor - a problem?

In [ ]:
trained_model = train_model(test_X_train, test_y_train, model=model.features)

# Dataframe stuff


In [ ]:
os.chdir('../HumBugDB/notebooks')

In [ ]:
df = pd.read_csv(config.data_df)
idx_multiclass = np.logical_and(df['country'] == 'Tanzania', df['location_type'] == 'cup')
df_all = df[idx_multiclass]
classes = ['an arabiensis','culex pipiens complex', 'ae aegypti','an funestus ss','an squamosus',
               'an coustani','ma uniformis','ma africanus']

In [ ]:
df_all = df_all.iloc[::50,:]

In [ ]:
# copied from species_classification
import collections
species_dict = collections.OrderedDict()
species_recordings = collections.OrderedDict()
for species in classes:
    # Number of total audio clips per species (includes repeats from same filename)
    species_recordings[species] = len(pd.unique(df_all[df_all.species==species].name)) # Number of unique audio recordings (and hence mosquitoes)
    species_dict[species] = sum(df_all[df_all.species==species].length)

In [ ]:
# Divide recordings into train and test, with recording shuffling fixed by random_state
train_fraction = 0.75
train_recordings = {}
test_recordings = {}

for i in range(len(classes)):
    n_train = int(species_recordings[classes[i]] * train_fraction)
    n_test = species_recordings[classes[i]] - n_train
    print(classes[i], n_train, n_test)
    df_class = df_all[df_all.species == classes[i]]
    # !!!I think it is probably best to shuffle these to mimic W but need to make sure each ID of mosquito is unique per recording
    train_recordings[i] =  shuffle(pd.unique(df_class.name), random_state=42)[:n_train]  
    test_recordings[i] = shuffle(pd.unique(df_class.name),random_state=42)[n_train:]

In [ ]:
def get_vggish_feats(df_all, label_recordings_dict, data_dir):
  X, y = [], []
  for class_label in tqdm(label_recordings_dict.keys(), position=0, leave=True): # Loop over classes
    for i in label_recordings_dict[class_label]: # Loop over recordings in class
      df_match = df_all[df_all.name == i]
      for idx, row in df_match.iterrows(): # Loop over clips in recording
        _, file_format = os.path.splitext(row['name'])
        filename = os.path.join(data_dir, str(row['id']) + file_format)
        feat = wavfile_to_examples(filename)
        # if config.norm_per_sample:
          # feat = (feat-np.mean(feat))/np.std(feat)           
        X.append(feat)
        y.append(class_label)
  return X, y


In [ ]:
X_test, y_test = get_vggish_feats(df_all, test_recordings, config.data_dir)

In [ ]:
X_train, y_train = get_vggish_feats(df_all, train_recordings, config.data_dir)

In [ ]:
X_train[0].shape

In [ ]:
# bad idea. need to preserve labels too
X_test = torch.cat(X_test, dim=0)
X_train = torch.cat(X_train, dim=0)

In [ ]:
print(X_test.shape, X_train.shape)

In [ ]:
model.forward()

In [ ]:
from vggish import VGG
vgg_model = VGG

In [ ]:
vgg_model.forward()

In [ ]:
from vggish import VGGish
model_vggish = VGGish({'vggish':'www', 1:'xxx'})